### **Anime Analytics**

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import mysql.connector

**Conexión a la base de datos**

In [2]:
try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='saito17Jr',
        database='anime'
    )

    if connection.is_connected():
        db_info = connection.get_server_info()
        print(f'Conexión exitosa a MySQL versión {db_info}')

except mysql.connector.Error as e:
    print(f'Error al conectar a MySQL {e}')

Conexión exitosa a MySQL versión 8.0.44


C:\Users\52771\AppData\Local\Temp\ipykernel_2032\1022429942.py:10: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = connection.get_server_info()
